In [1]:
# imports
import logging
import sys
import time
from tf_pose import common
import cv2
import numpy as np
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh
import matplotlib.pyplot as plt
import asyncio
import datetime
import random
import websockets
import nest_asyncio
import http.server
import socketserver
import threading
from IPython.display import IFrame
import base64
from PIL import Image
from io import BytesIO
import re
from utils import run_websocket_server, run_http_server


nest_asyncio.apply()

logger = logging.getLogger('TfPoseEstimatorRun')
logger.handlers.clear()
logger.setLevel(logging.ERROR)
ch = logging.StreamHandler()
ch.setLevel(logging.ERROR)
formatter = logging.Formatter('[%(asctime)s] [%(name)s] [%(levelname)s] %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




In [2]:
# params 656x368 cmu (trained in 656x368)
w, h = 432, 368
image_path = './data/images/daniela.jpg'
model = 'cmu'
resize_out_ratio = 4.0

In [3]:
e = TfPoseEstimator(get_graph_path(model), target_size=(w, h))

[2019-12-27 18:03:58,744] [TfPoseEstimator] [INFO] loading graph from /home/ax/Programs/tf-pose-estimation/models/graph/cmu/graph_opt.pb(default size=656x368)
2019-12-27 18:03:58,744 INFO loading graph from /home/ax/Programs/tf-pose-estimation/models/graph/cmu/graph_opt.pb(default size=656x368)


2019-12-27 18:03:58,747 WARNING From /home/ax/Programs/tf-pose-estimation/tf_pose/estimator.py:311: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



2019-12-27 18:03:58,751 WARNING From /home/ax/Programs/tf-pose-estimation/tf_pose/estimator.py:312: The name tf.GraphDef is deprecated. Please use tf.compat.v1.GraphDef instead.



2019-12-27 18:03:59,110 WARNING From /home/ax/Programs/tf-pose-estimation/tf_pose/estimator.py:330: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



2019-12-27 18:03:59,445 WARNING From /home/ax/Programs/tf-pose-estimation/tf_pose/estimator.py:332: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



...

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


2019-12-27 18:03:59,546 WARNING From /home/ax/Programs/tf-pose-estimation/tf_pose/estimator.py:352: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


2019-12-27 18:03:59,548 WARNING From /home/ax/Programs/tf-pose-estimation/tf_pose/estimator.py:357: The name tf.variables_initializer is deprecated. Please use tf.compat.v1.variables_initializer instead.



2019-12-27 18:03:59,548 WARNING From /home/ax/Programs/tf-pose-estimation/tf_pose/estimator.py:358: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



2019-12-27 18:03:59,549 WARNING From /home/ax/Programs/tf-pose-estimation/tf_pose/estimator.py:360: The name tf.report_uninitialized_variables is deprecated. Please use tf.compat.v1.report_uninitialized_variables instead.



In [4]:
def base64_str_2_numpy(b64_image_str):
    imgstr = re.search(r'base64,(.*)', b64_image_str).group(1)
    buffer = base64.b64decode(imgstr) # decode b64 to buffer
    pil_image = Image.open(BytesIO(buffer)) 
    return np.array(pil_image)

def numpy_2_base64(numpy_image):
    # res = cv2.cvtColor(numpy_image.astype(np.uint8), cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(numpy_image)
    #get image data as byte
    buffer = BytesIO()
    pil_image.save(buffer, format='png')
    buffer_value = buffer.getvalue()

    #byte to string
    return base64.b64encode(buffer_value)

async def websocket_callback(websocket, path):
    while True:
        b64_image_str = await websocket.recv() # read image from websocket
        print('Received an image')
        
        test_image = base64_str_2_numpy(b64_image_str)
        
        humans = e.inference(test_image, resize_to_default=(w > 0 and h > 0), upsample_size=resize_out_ratio)

        image_result = TfPoseEstimator.draw_humans(test_image, humans, imgcopy=True)
        image_result_b64 = numpy_2_base64(image_result)
        print('Sending result')
        await websocket.send(image_result_b64)

In [5]:
# port of the websocket server
port = 1236
threading \
    .Thread(target=run_websocket_server, args=(websocket_callback, port)) \
    .start()


Starting websocket server on port 1236


In [6]:
threading \
    .Thread(target=run_http_server) \
    .start()

serving at port 8080


In [7]:
IFrame(src=f'http://0.0.0.0:8080/html/uploader.html?port={port}', width=1500, height=800)

Received an image
Sending result
Received an image
Sending result
Received an image
Sending result
